In [0]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Ejercicios RDD").getOrCreate()

sc = spark.sparkContext

print(sc)

conf = SparkConf().setMaster("local[*]").setAppName("Ejercicios RDD")

conf.set("spark.hadoop.fs.defaultFS", "file:///")
conf.set("spark.hadoop.io.file.buffer.size", "4096")


<SparkContext master=local[8] appName=Databricks Shell>
Out[11]: <pyspark.conf.SparkConf at 0x7efdcff79dc0>

## Ejercicio 1. Operaciones con RDD’s

- Crea un RDD con números del 1 al 100. Calcula la cantidad de elementos, la suma total, el promedio y el valor máximo. Imprime los resultados.
- Sobre el RDD anterior, filtra únicamente los números pares. Usa map para transformar cada número filtrado en su cuadrado. Muestra los primeros 10 resultados.
- Transforma el RDD anterior en un RDD de tuplas en formato (categoría, valor), donde la categoría sea el último dígito del cuadrado convertido a string. Utiliza reduceByKey para sumar los valores de cada categoría. Imprime el resultado para cada categoría.


In [0]:
rdd = sc.parallelize(range(1, 101))

cantidad = rdd.count()
suma_total = rdd.sum()
promedio = suma_total / cantidad
valor_maximo = rdd.max()

print(f"Cantidad de elementos: {cantidad}")
print(f"Suma total: {suma_total}")
print(f"Promedio: {promedio}")
print(f"Valor máximo: {valor_maximo}")



rdd_pares = rdd.filter(lambda x: x % 2 == 0)
rdd_cuadrados = rdd_pares.map(lambda x: x ** 2)
primeros_10 = rdd_cuadrados.take(10)

print("Primeros 10 cuadrados de números pares:", primeros_10)


rdd_tuplas = rdd_cuadrados.map(lambda x: (str(x % 10), x))
rdd_sumado = rdd_tuplas.reduceByKey(lambda x, y: x + y)
resultado = rdd_sumado.collect()

print("Suma de valores por categoría (último dígito del cuadrado):")
for categoria, suma in sorted(resultado):
    print(f"Dígito {categoria}: {suma}")


Cantidad de elementos: 100
Suma total: 5050
Promedio: 50.5
Valor máximo: 100
Primeros 10 cuadrados de números pares: [4, 16, 36, 64, 100, 144, 196, 256, 324, 400]
Suma de valores por categoría (último dígito del cuadrado):
Dígito 0: 38500
Dígito 4: 66680
Dígito 6: 66520


## Ejercicio 2. Operaciones con dataframes
- Sobre los datos de ratings (u.data), películas (u.item) y usuarios (u.user), muestra la película con mayor número de votos positivos (4,5) según los siguientes criterios:
  - Sexo
  - Edad (0-25, 25-45, 45-65, >65)
  - Ocupación

In [0]:
ratings_df = spark.read.csv("/FileStore/u.data", sep="\t", inferSchema=True)
ratings_df = ratings_df.toDF("user_id", "movie_id", "rating", "timestamp")

movies_df = spark.read.csv("/FileStore/u.item", sep="|", inferSchema=True, encoding="ISO-8859-1")
movies_df = movies_df.toDF("movie_id", "title", "release_date", "video_release_date", "IMDb_URL", 
                            "unknown", "Action", "Adventure", "Animation", "Children", "Comedy", "Crime", 
                            "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", 
                            "Romance", "Sci-Fi", "Thriller", "War", "Western")

users_df = spark.read.csv("/FileStore/u.user", sep="|", inferSchema=True)
users_df = users_df.toDF("user_id", "age", "gender", "occupation", "zip_code")


positive_ratings_df = ratings_df.filter(col("rating") >= 4)



ratings_movies_df = positive_ratings_df.join(movies_df, "movie_id")
ratings_users_df = ratings_movies_df.join(users_df, "user_id")


most_voted_by_gender = ratings_users_df.groupBy("title", "gender").agg(count("rating").alias("count"))
window_spec = Window.partitionBy("gender").orderBy(col("count").desc())
top_movies_by_gender = most_voted_by_gender.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1)

top_movies_by_gender.select("gender", "title", "count").show()


from pyspark.sql.functions import when

ratings_users_df = ratings_users_df.withColumn(
    "age_group",
    when(col("age") <= 25, "0-25")
    .when((col("age") > 25) & (col("age") <= 45), "25-45")
    .when((col("age") > 45) & (col("age") <= 65), "45-65")
    .otherwise(">65")
)

most_voted_by_age = ratings_users_df.groupBy("title", "age_group").agg(count("rating").alias("count"))
window_spec_age = Window.partitionBy("age_group").orderBy(col("count").desc())

top_movies_by_age = most_voted_by_age.withColumn("rank", row_number().over(window_spec_age)).filter(col("rank") == 1)

top_movies_by_age.select("age_group", "title", "count").show()


most_voted_by_occupation = ratings_users_df.groupBy("title", "occupation").agg(count("rating").alias("count"))
window_spec_occ = Window.partitionBy("occupation").orderBy(col("count").desc())

top_movies_by_occupation = most_voted_by_occupation.withColumn("rank", row_number().over(window_spec_occ)).filter(col("rank") == 1)

top_movies_by_occupation.select("occupation", "title", "count").show()


+------+----------------+-----+
|gender|           title|count|
+------+----------------+-----+
|     F|Star Wars (1977)|  121|
|     M|Star Wars (1977)|  380|
+------+----------------+-----+

+---------+----------------+-----+
|age_group|           title|count|
+---------+----------------+-----+
|     0-25|Star Wars (1977)|  151|
|    25-45|Star Wars (1977)|  265|
|    45-65|    Fargo (1996)|   90|
|      >65|     Emma (1996)|    4|
+---------+----------------+-----+

+-------------+--------------------+-----+
|   occupation|               title|count|
+-------------+--------------------+-----+
|administrator|    Star Wars (1977)|   36|
|       artist|    Star Wars (1977)|   16|
|       doctor|English Patient, ...|    5|
|     educator|        Fargo (1996)|   46|
|     engineer|    Star Wars (1977)|   47|
|entertainment|      Contact (1997)|   11|
|    executive|      Contact (1997)|   16|
|   healthcare|      Titanic (1997)|    9|
|    homemaker|   Saint, The (1997)|    5|
|       la

## Ejercicio 3. Análisis de ventas
Disponemos de tres ficheros de ventas de una tienda americana de productos de tecnología, correspondientes a los meses de octubre, noviembre y diciembre. Realiza las siguientes tareas:
- Lectura de los ficheros como RDDs
- Limpieza de datos: eliminar registros vacíos o con un número de campos incorrecto. Detectar y eliminar posibles filas de cabecera (header) si aparecen como registros.
- Convertir a dataframe con el siguiente esquema:
  - "Order ID", Integer
  - "Product", String
  - "Quantity Ordered", Integer
  - "Price Each", Double
  - "Order Date", String
  - "Purchase Address", String
- Unir los registros en un único dataframe "Ventas"
- Generar una tabla temporal "Productos" con los productos vendidos y su precio medio
- Crear campos adicionales en el dataframe Ventas: year, month, state, city, CP a partir de los campos Order Date y Purchase Address
- Obtener el día con mayores ingresos
- Obtener el producto más vendido (por cantidad total) y qué ingresos ha generado en total.
- Listar las 10 ciudades con mayores ventas (en ingresos).
- Tabla de número de pedidos e importe por horas (campo hour extraído de Order Date).
- Almacenar los datos en formato Parquet en la carpeta "sales", particionando por year y month.

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-647388550127399>:54
     44 df_ventas = rdd_limpio.map(lambda x: Row(
     45     int(x[0]), 
     46     x[1], 
   (...)
     50     x[5]
     51 )).toDF(schema)
     53 # Mostrar datos
---> 54 df_ventas.show(5)
     56 ###
     58 from pyspark.sql.functions import avg

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
 

## Ejercicio 4. Completar Problema del Viajante con Búsqueda en Anchura en PySpark
Se dispone de un diccionario de distancias entre 10 ciudades españolas. El objetivo es implementar, utilizando PySpark, un algoritmo que resuelva un caso simplificado del Problema del Viajante (TSP). Específicamente, se debe encontrar la ruta de menor distancia que parta desde Madrid y que visite cada una de las ciudades exactamente una vez.

Para ello, se utilizará un recorrido en anchura (Breadth First Search, BFS) sobre un RDD. Cada ruta parcial se representará como una tupla formada por:
- Una lista con el orden de ciudades visitadas.
- La distancia acumulada en esa ruta.

Se empleará una variable broadcast para compartir la tabla de distancias entre los workers, evitando así que cada tarea tenga que reenviar estos datos.

Entender las funciones definidas, completar las llamadas que faltan y ejecutar.

In [0]:
# Diccionario que contiene las distancias (en kilómetros) entre cada par de ciudades.
distancias = {
    "Madrid": {"Barcelona": 620, "Valencia": 350, "Sevilla": 530, "Zaragoza": 320, "Málaga": 530, "Bilbao": 400, "Murcia": 420, "Palma": 500, "Alicante": 420},
    "Barcelona": {"Madrid": 620, "Valencia": 350, "Sevilla": 1000, "Zaragoza": 300, "Málaga": 1080, "Bilbao": 600, "Murcia": 700, "Palma": 210, "Alicante": 520},
    "Valencia": {"Madrid": 350, "Barcelona": 350, "Sevilla": 660, "Zaragoza": 300, "Málaga": 600, "Bilbao": 700, "Murcia": 140, "Palma": 250, "Alicante": 170},
    "Sevilla": {"Madrid": 530, "Barcelona": 1000, "Valencia": 660, "Zaragoza": 700, "Málaga": 210, "Bilbao": 750, "Murcia": 500, "Palma": 650, "Alicante": 450},
    "Zaragoza": {"Madrid": 320, "Barcelona": 300, "Valencia": 300, "Sevilla": 700, "Málaga": 670, "Bilbao": 300, "Murcia": 510, "Palma": 450, "Alicante": 500},
    "Málaga": {"Madrid": 530, "Barcelona": 1080, "Valencia": 600, "Sevilla": 210, "Zaragoza": 670, "Bilbao": 800, "Murcia": 400, "Palma": 700, "Alicante": 480},
    "Bilbao": {"Madrid": 400, "Barcelona": 600, "Valencia": 700, "Sevilla": 750, "Zaragoza": 300, "Málaga": 800, "Murcia": 650, "Palma": 750, "Alicante": 680},
    "Murcia": {"Madrid": 420, "Barcelona": 700, "Valencia": 140, "Sevilla": 500, "Zaragoza": 510, "Málaga": 400, "Bilbao": 650, "Palma": 350, "Alicante": 75},
    "Palma": {"Madrid": 500, "Barcelona": 210, "Valencia": 250, "Sevilla": 650, "Zaragoza": 450, "Málaga": 700, "Bilbao": 750, "Murcia": 350, "Alicante": 520},
    "Alicante": {"Madrid": 420, "Barcelona": 520, "Valencia": 170, "Sevilla": 450, "Zaragoza": 500, "Málaga": 480, "Bilbao": 680, "Murcia": 75, "Palma": 520}
}

# Variable “distancias_compartidas” de solo lectura, distribuida entre nodos del cluster
distancias_compartidas = ## COMPLETAR ##

# Extiende cada ruta parcial añadiendo un nuevo nodo para cada ciudad no visitada.
def expandir_camino(camino):
    ruta = camino[0]
    distancia = camino[1]
    ultimo_nodo = ruta[-1]
    caminos_expandidos = []
    
    nodos_visitados = set(ruta)
    nodos_no_visitados = set(distancias_compartidas.value.keys()) - nodos_visitados
    
    for nodo in nodos_no_visitados:
        nueva_ruta = ruta + [nodo]
        nueva_distancia = distancia + distancias_compartidas.value[ultimo_nodo][nodo]
        caminos_expandidos.append((nueva_ruta, nueva_distancia))
    
    return caminos_expandidos

# Para cada par de caminos, se queda con el de menor distancia
def reducir_caminos(camino1, camino2):
    return camino1 if camino1[1] < camino2[1] else camino2

# Inicializa el camino con el nodo inicial y la distancia 0
nodo_inicial = ['Madrid']
distancia = 0
camino_rdd = ## COMPLETAR ##

# Iterar hasta que todos los nodos sean visitados por todos los caminos
while len(camino_rdd.first()[0]) < len(distancias_compartidas.value):
    camino_rdd = ## COMPLETAR ##

# Ver número de caminos generados
num_caminos = ## COMPLETAR ##
print(f"Se han generado {num_caminos} caminos")

# Obtener el camino más corto
camino_mas_corto = ## COMPLETAR ##

print(f"El camino más corto es: {camino_mas_corto[0]} con una distancia de {camino_mas_corto[1]} km")
